# Sentiment Analysis using the emotion dataset with 6 emotions
using HuggingFace Trainer

## Get the emotion dataset

In [ ]:
from datasets import load_dataset

ds = load_dataset("emotion")
ds

In [ ]:
classes={}

for i, mood in enumerate(ds['train'].features['label'].names):
    classes[i]=mood    
classes

## Let's see examples of each emotion

In [ ]:
mood=0 

for i in range (100):
    if ds['train']['label'][i] ==mood:
        s=ds['train']['text'][i]
        print(f'{classes[mood]}: {s}  ')
        mood+=1
        if mood==6:
            break

## Preprocessing the dataset to make it compatible with model's tokenizer

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_fn(sample):
    return tokenizer(sample['text'], truncation=True)

tokenized_ds = ds.map(tokenize_fn, batched=True)

## Using DataCollator for dynamic padding

### DataCp;;ator provides dynamic padding. Wehn combined with dataloader, the amount of padding to use depends on the longest sentence in the current batch. This only works when using GPU, and pytorch

In [ ]:
from torch.utils.data import DataLoader

collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Importing the model

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=len(classes), id2label=classes)
model.config.id2label

## Hyper-Parameters

In [ ]:
import math 

lr=1e-4
epochs=10
batchSize=128
totalSteps=math.ceil(ds['train'].num_rows/batchSize)*epochs
print(f'training has {totalSteps} steps')

## importing optimizer and learning rate scheduler

In [ ]:
from transformers import AdamW
from transformers import get_scheduler

opt=AdamW(model.parameters(), lr=lr)
scheduler = get_scheduler("cosine",optimizer=opt,num_warmup_steps=0,num_training_steps=totalSteps)

## Training the model using the Trainer and TrainingArgument

In [ ]:
from datasets import load_metric 
import numpy as np 

metric=load_metric('accuracy')

def compute_metrics(samples):
    preds, lbls = samples
    preds = np.argmax(preds, axis=1)    
    return metric.compute(predictions=preds, references=lbls)

In [ ]:
from transformers import TrainingArguments, Trainer 

args = TrainingArguments(
    "results",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    # learning_rate=lr,
    per_device_train_batch_size=batchSize,
    per_device_eval_batch_size=batchSize,
    num_train_epochs=epochs,
    weight_decay=0.01,        
    load_best_model_at_end=True,
    logging_dir='logs',            # directory for storing logs
    logging_steps=50    
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['validation'],
    tokenizer=tokenizer,
    data_collator=collator,    
    optimizers=(opt, scheduler),
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('trained')

In [ ]:
trainer.train('trained')

## Evaluating the trained model

In [ ]:
trainer.evaluate()

## Hyper Parameters tuning using optuna

In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=len(classes) , id2label=classes)

In [ ]:
trainer = Trainer(    
    model_init=model_init,
    args=args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['validation'],
    tokenizer=tokenizer,
    data_collator=collator,    
    compute_metrics=compute_metrics
)

## Start the Hyperparameters search
### Trainer will automatically use optuna to do the tuning

In [ ]:
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

## Tuning result

In [ ]:
best_run

## Set the tuned parameters iunto trainer and train again

In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

## Saving the trained model

In [ ]:
dir="saved-model"

model.save_pretrained(dir)

## Inferencing

### load the trained model

In [ ]:
from transformers import AutoModelForSequenceClassification
model2=AutoModelForSequenceClassification.from_pretrained(dir)

In [ ]:
import torch 
device=torch.device('cuda' if torch.cuda.is_available() else "cpu")
model2.to(device)

In [ ]:
def getMood(samples):
    input=tokenizer(samples,padding=True, truncation=True, return_tensors="pt" ).to(device)
    out=model2(**input)
    res=torch.argmax(out.logits , dim=1)    
    return res.cpu()

In [ ]:
str1="i am really annoyed"
str2="my heart yearns for her return"
str3="my heart is torn apart"
str4="it's a great sunday today"
str5="i'm left speechless"
str6="i stepped into the unknown"

samples=[str1, str2, str3, str4, str5,str6]
res=getMood(samples)

for i, r in enumerate(res):
    print(f'{samples[i]}: {model2.config.id2label[r.item() ]} ')